## So sánh mã hóa biến hạng mục

Ở phần này, chúng ta sẽ so sánh chất lượng củ các kỹ thuật mã hóa hạng mục đặc trưng đã học.

Chúng ta sẽ so sánh:

- Mã hóa one-hot
- Thay các nhãn bằng số đếm
- Nhãn sắp xếp theo mục tiêu
- Mã hóa trung bình
- WoE

Sử dụng tập dữ liệu Titanic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [3]:
# load tập dữ liệu titanic

# chỉ sử dụng các cột này để mô phỏng
cols = ['pclass', 'age', 'sibsp', 'parch', 'fare',
        'sex', 'cabin', 'embarked', 'survived']

data = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Bai tap lap/Mon_hoc_2/titanic.csv', usecols=cols)

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [4]:
# kiểm tra dữ liệu bị khuyết

data.isnull().sum()

pclass         0
survived       0
sex            0
age          263
sibsp          0
parch          0
fare           1
cabin       1014
embarked       2
dtype: int64

In [5]:
# loại các quan sát có NA trong fare và embarked

data.dropna(subset=['fare', 'embarked'], inplace=True)

In [6]:
# trích xuất chữ cái đầu của cabin

data['cabin'] = data['cabin'].astype(str).str[0]

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B,S
1,1,1,male,0.9167,1,2,151.5500,C,S
2,1,0,female,2.0000,1,2,151.5500,C,S
3,1,0,male,30.0000,1,2,151.5500,C,S
4,1,0,female,25.0000,1,2,151.5500,C,S


In [7]:
# loại các quan sát có cabin = T, chúng quá it

data = data[data['cabin'] != 'T']

In [8]:
# chia thành tập huấn luyện và tập kiểm tra

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels='survived', axis=1),  # các yếu tố dự báo
    data['survived'],  # mục tiêu
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((913, 8), (392, 8))

In [10]:
# thay các giá trị null trong biến số bằng mean


def impute_na(df, variable, value):
    df[variable].fillna(value, inplace=True)


impute_na(X_test, 'age', X_train['age'].mean())
impute_na(X_train, 'age',  X_train['age'].mean())
# lưu ý cách gán trước tiên trong tập kiểm tra, theo cách này giá trị của
# được dùng tương tự với cả tập huấn luyện và tập kiểm tra

In [ ]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,female,30.00000,1,0,13.8583,n,C
698,3,male,18.00000,0,0,8.6625,n,S
1291,3,male,29.79847,0,0,8.7125,n,S
1229,3,male,27.00000,0,0,8.6625,n,S
118,1,male,29.79847,0,0,26.5500,D,S


In [11]:
# kiểm tra xem còn gái trị bị khuyết sau khi gán NA không

X_train.isnull().sum(), X_test.isnull().sum()

(pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64,
 pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64)

### Mã hóa one-hot

In [12]:
# Yêu cầu 1:
def get_OHE(df):
    # Mã hóa one-hot
    ## VIẾT CODE Ở ĐÂY:
    df_OHE = pd.concat(
        [df[['pclass', 'age', 'sibsp', 'parch', 'fare']],
         pd.get_dummies(df[['sex', 'cabin', 'embarked']], drop_first=True)],
        axis=1)

    return df_OHE

## VIẾT CODE Ở ĐÂY:
X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

X_train_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
402,2,30.00000,1,0,13.8583,0,0,0,0,0,0,0,1,0,0
698,3,18.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
1291,3,29.79847,0,0,8.7125,1,0,0,0,0,0,0,1,0,1
1229,3,27.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
118,1,29.79847,0,0,26.5500,1,0,0,1,0,0,0,0,0,1


In [13]:
X_test_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
586,2,29.00000,1,0,26.0000,0,0,0,0,0,0,0,1,0,1
200,1,46.00000,0,0,75.2417,1,0,1,0,0,0,0,0,0,0
831,3,40.00000,1,6,46.9000,1,0,0,0,0,0,0,1,0,1
1149,3,29.79847,0,0,7.7208,0,0,0,0,0,0,0,1,1,0
393,2,25.00000,0,0,31.5000,1,0,0,0,0,0,0,1,0,1


### Mã hóa đếm

In [14]:
# Yêu cầu 2
def categorical_to_counts(df_train, df_test):

    # tạo bản sao tạm thời của các dataframe ban đầu
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy()

    for col in ['sex', 'cabin', 'embarked']:

        # tạo dictionary mapping để đếm
        ## VIẾT CODE Ở ĐÂY:
        counts_map = df_train_temp[col].value_counts().to_dict()

        # ánh xạ lại các nhãn tới count
        ## VIẾT CODE Ở ĐÂY:
        df_train_temp[col] = df_train_temp[col].map(counts_map)
        df_test_temp[col] = df_test_temp[col].map(counts_map)

    return df_train_temp, df_test_temp

## VIẾT CODE Ở ĐÂY:
X_train_count, X_test_count = categorical_to_counts(X_train, X_test)

X_train_count.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,326,30.00000,1,0,13.8583,702,184
698,3,587,18.00000,0,0,8.6625,702,647
1291,3,587,29.79847,0,0,8.7125,702,647
1229,3,587,27.00000,0,0,8.6625,702,647
118,1,587,29.79847,0,0,26.5500,33,647


### Mã hóa số nguyên đã xếp

In [ ]:
df_temp = X_train

In [17]:
# Yêu cầu 3
def categories_to_ordered(df_train, df_test, y_train, y_test):

    # tạo bản sao tạm thời của tập dữ liệu
    ## VIẾT CODE Ở ĐÂY:
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # sắp xếp các hạng mục theo trung bình mục tiêu
        ## VIẾT CODE Ở ĐÂY:
        ordered_labels = df_train_temp.groupby([col])['survived'].mean().sort_values().index

        # tạo dictionary để ánh xạ các nhãn đã sắp xếp tới số thứ tự
        ## VIẾT CODE Ở ĐÂY:
        ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

        # ánh xạ lại các hạng mục tới các số thứ tự
        ## VIẾT CODE Ở ĐÂY:
        df_train_temp[col] = df_train[col].map(ordinal_label)
        df_test_temp[col] = df_test[col].map(ordinal_label)

    # loại bỏ mục tiêu
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ordered, X_test_ordered = categories_to_ordered(
    X_train, X_test, y_train, y_test)

X_train_ordered.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,1,30.00000,1,0,13.8583,0,2
698,3,0,18.00000,0,0,8.6625,0,0
1291,3,0,29.79847,0,0,8.7125,0,0
1229,3,0,27.00000,0,0,8.6625,0,0
118,1,0,29.79847,0,0,26.5500,5,0


### Mã hóa trung bình

In [18]:
# Yêu cầu 4
def categories_to_mean(df_train, df_test, y_train, y_test):

    # tạo bản sao tạm thời của tập dữ liệu
    ## VIẾT CODE Ở ĐÂY:
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # tính mục tiêu trung bình trên mỗi hạng mục
        ## VIẾT CODE Ở ĐÂY:
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().to_dict()

        # ánh xạ lại các hạng mục tới mục tiêu trung bình
        ## VIẾT CODE Ở ĐÂY:
        df_train_temp[col] = df_train[col].map(ordered_labels)
        df_test_temp[col] = df_test[col].map(ordered_labels)

    # loại bỏ mục tiêu
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp

## VIẾT CODE Ở ĐÂY:
X_train_mean, X_test_mean = categories_to_mean(
    X_train, X_test, y_train, y_test)

X_train_mean.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.730061,30.00000,1,0,13.8583,0.292023,0.516304
698,3,0.173765,18.00000,0,0,8.6625,0.292023,0.332303
1291,3,0.173765,29.79847,0,0,8.7125,0.292023,0.332303
1229,3,0.173765,27.00000,0,0,8.6625,0.292023,0.332303
118,1,0.173765,29.79847,0,0,26.5500,0.696970,0.332303


### Mã hóa tỷ lệ xác suất

In [19]:
# Yêu cầu 5
def categories_to_ratio(df_train, df_test, y_train, y_test):

    # tạo bản sao tạm thời của tập dữ liệu
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # tạo df chứa các phần khác nhau của phương trình WoE
        # xác suất survived =1
        ## VIẾT CODE Ở ĐÂY:
        prob_df = pd.DataFrame(df_train_temp.groupby([col])['survived'].mean())

        # xác suất survived = 0
        ## VIẾT CODE Ở ĐÂY:
        prob_df['died'] = 1 - prob_df['survived']

        # tính WoE
        ## VIẾT CODE Ở ĐÂY:
        prob_df['Ratio'] = np.log(prob_df['survived']/prob_df['died'])

        # ghi woe trong dictionary
        ## VIẾT CODE Ở ĐÂY:
        woe = prob_df['Ratio'].to_dict()

        # ánh xạ lại các nhãn tới WoE
        ## VIẾT CODE Ở ĐÂY:
        df_train_temp[col] = df_train[col].map(woe)
        df_test_temp[col] = df_test[col].map(woe)

    # loại mục tiêu
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp

## VIẾT CODE Ở ĐÂY:
X_train_ratio, X_test_ratio = categories_to_ratio(X_train, X_test, y_train, y_test)

X_train_ratio.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.994934,30.00000,1,0,13.8583,-0.885580,0.065241
698,3,-1.559176,18.00000,0,0,8.6625,-0.885580,-0.697788
1291,3,-1.559176,29.79847,0,0,8.7125,-0.885580,-0.697788
1229,3,-1.559176,27.00000,0,0,8.6625,-0.885580,-0.697788
118,1,-1.559176,29.79847,0,0,26.5500,0.832909,-0.697788


### Chất lượng rừng ngẫu nhiên

In [20]:
# tạo một hàm xây dựng rừng ngẫu nhiên và so sánh chất lượng trong tập huấn luyện và tập kiểm tra


def run_randomForests(X_train, X_test, y_train, y_test):

    rf = RandomForestClassifier(n_estimators=50, random_state=39, max_depth=3)
    rf.fit(X_train, y_train)

    print('Train set')
    pred = rf.predict_proba(X_train)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = rf.predict_proba(X_test)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [21]:
# OHE
run_randomForests(X_train_OHE, X_test_OHE, y_train, y_test)

Train set
Random Forests roc-auc: 0.8488938507340109
Test set
Random Forests roc-auc: 0.8072730715135779


In [22]:
# đếm
run_randomForests(X_train_count, X_test_count, y_train, y_test)

Train set
Random Forests roc-auc: 0.8654552920644698
Test set
Random Forests roc-auc: 0.8194309206967434


In [23]:
# các nhãn đã sắp xếp
run_randomForests(X_train_ordered, X_test_ordered, y_train, y_test)

Train set
Random Forests roc-auc: 0.8669027820552304
Test set
Random Forests roc-auc: 0.8219733852645245


In [24]:
# mã hóa trung bình
run_randomForests(X_train_mean, X_test_mean, y_train, y_test)

Train set
Random Forests roc-auc: 0.867010573863053
Test set
Random Forests roc-auc: 0.8207562479714378


In [25]:
# tỷ lệ
run_randomForests(X_train_ratio, X_test_ratio, y_train, y_test)

Train set
Random Forests roc-auc: 0.867010573863053
Test set
Random Forests roc-auc: 0.8207562479714378


Khi so sánh các giá trị roc_auc trên các tập kiểm tra, chúng ta thấy rằng mã hóa one-hot có chất lượng kém hơn. Điều này có ý nghĩa vì cây không hoạt động tốt trong tập dữ liệu có không gian đặc trưng lớn.

Các mã hóa còn lại cho chất lượng tương tự. Điều này cũng có ý nghĩa, vì cây là mô hình phi tuyến tính nên các mã hóa hướng dẫn có mục tiêu có thể sẽ không cải thiện chất lượng mô hình

### Chất lượng hồi quy Logistic

In [26]:
def run_logistic(X_train, X_test, y_train, y_test):

    # hàm huấn luyện và kiểm tra chất lượng của hồi quy Logistic
    logit = LogisticRegression(random_state=44, C=0.01, max_iter=100)
    logit.fit(X_train, y_train)

    print('Train set')
    pred = logit.predict_proba(X_train)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Test set')
    pred = logit.predict_proba(X_test)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [32]:
# OHE
run_logistic(X_train_OHE, X_test_OHE, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8287932450467097
Test set
Logistic Regression roc-auc: 0.8013902412636589


In [28]:
# số đếm
run_logistic(X_train_count, X_test_count, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7898188071039933
Test set
Logistic Regression roc-auc: 0.7387482419127988


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
# các nhãn đã sắp xếp
run_logistic(X_train_ordered, X_test_ordered, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8223924648393389
Test set
Logistic Regression roc-auc: 0.8006870063832089


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
# mã hóa trung bình
run_logistic(X_train_mean, X_test_mean, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7791217534134072
Test set
Logistic Regression roc-auc: 0.7481878178080709


In [33]:
# tỷ lệ
run_logistic(X_train_ratio, X_test_ratio, y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8508546350477364
Test set
Logistic Regression roc-auc: 0.8204857730174184


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Với hồi quy Logistic, mã hóa one-hot cho chất lượng tốt nhất vì nó bảo toàn các mối quan hệ tuyến tính với các biến và mục tiêu, cũng như với trọng số bằng chứng và mã hóa có sắp xếp.

Tuy nhiên, hãy lưu ý cách mã hóa đếm trả về chất lượng kém hơn vì nó không tạo ra mối quan hệ đơn điệu giữa các biến và mục tiêu; trong trường hợp này, mã hóa mục tiêu trung bình thường gây ra overfitting.